In [10]:
import pandas as pd
import polars as pl
import fun
import random
import numpy as np
import math

In [11]:
np.random.beta(1,30)*80


0.2512489575732717

## Data Prep

In [34]:
customer_lat = -6.2124234
customer_long = 106.816634

coord_center = (customer_long, customer_lat)
N0 = 10000

In [99]:
df0 = fun.dummy_dataset(2,8)
df1 = fun.dummy_dataset(15,3)
df2 = fun.dummy_dataset(80,6)
df3 = fun.dummy_dataset(80,100)
df4 = fun.dummy_dataset(120,200)

# df3 = fun.dummy_dataset(80,5)
# df4 = fun.dummy_dataset(80,15)
# df5 = fun.dummy_dataset(80,50)

In [36]:
df0

,long_array,lat_array,n_point
0,[],[],0
1,[106.81767258698731],[-6.212379084386128],1


## Optimize the loop

In [70]:
def f1(df):
    distance_summary = [fun.haversine_distance_summary(
                    zip(df['long_array'].loc[i], df['lat_array'].loc[i]),
                    coord2 = coord_center) 
                for i in range(len(df))]

    df['min_dist_to_other_dest'] = [one_row[0] for one_row in distance_summary]
    df['avg_dist_to_other_dest'] = [one_row[1] for one_row in distance_summary]
    return df

def f2(df):
    df['hdist_summary'] = df.apply(lambda x: fun.hdist_summary(x['long_array'], x['lat_array'], coord_center), axis=1)
    df[['min_dist_to_other_dest', 'avg_dist_to_other_dest']] = pd.DataFrame(df['hdist_summary'].tolist(), index=df.index)

    return df

def f3(df):
    df['hdist_summary'] = df.apply(lambda x: fun.hdist_summary(x['long_array'], x['lat_array'], coord_center), axis=1)
    dsummary = df['hdist_summary'].tolist()
    df['min_dist_to_other_dest'] = [i for i,j in dsummary]
    df['avg_dist_to_other_dest'] = [j for i,j in dsummary]

    return df



In [69]:
%%timeit
f1(df0)

177 µs ± 933 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [71]:
%%timeit
f2(df0)

642 µs ± 41.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [72]:
%%timeit
f3(df0)

446 µs ± 25 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [73]:
%%timeit
f1(df1)

492 µs ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [74]:
%%timeit
f2(df1)

846 µs ± 14.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [75]:
%%timeit
f3(df1)

644 µs ± 27.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [83]:
%%timeit
f1(df2)

2.79 ms ± 50.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [84]:
%%timeit
f2(df2)

2.23 ms ± 97.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [85]:
%%timeit
f3(df2)

2.17 ms ± 188 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [80]:
%%timeit
f1(df3)

7.73 ms ± 87.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [81]:
%%timeit
f2(df3)

3.27 ms ± 110 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [82]:
%%timeit
f3(df3)

3.07 ms ± 206 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [89]:
%%timeit
f1(df4)

18.1 ms ± 86.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [90]:
%%timeit
f2(df4)

4.99 ms ± 145 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [91]:
%%timeit
f3(df4)

4.77 ms ± 116 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [106]:
%%timeit
df0['hdist_summary'] = df0.apply(lambda x: fun.hdist_summary(x['long_array'], x['lat_array'], coord_center), axis=1)
dsummary = df0['hdist_summary'].tolist()
df0['min_dist_to_other_dest'] = dsummary

415 µs ± 22.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [138]:
%%timeit
# dsummary = df0['hdist_summary'].tolist()
df0['min_dist_to_other_dest'] = [i for i,j in dsummary]
df0['avg_dist_to_other_dest'] = [j for i,j in dsummary]

57.2 µs ± 367 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [139]:
%%timeit
# dsummary = df0['hdist_summary'].values
df0.loc[:,'min_dist_to_other_dest'] = [i for i,j in dsummary]
df0.loc[:,'avg_dist_to_other_dest'] = [j for i,j in dsummary]

86.9 µs ± 673 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [130]:
from operator import itemgetter